In [39]:
hydropathy_index = {"R":-2.5,"K":-1.5,"D":-0.9,"Q":-0.85,"N":-0.78,
                    "E":-0.74,"H":-0.4,"S":-0.18,"T":-0.05,"P":0.12,
                    "Y":0.26,"C":0.29,"G":0.48,"A":0.62,"M":0.64,
                    "W":0.81,"L":1.1,"V":1.1, "F":1.2,"I":1.4} 

occurance = {"A":8.76,"R":5.78,"N":3.93,"D":5.49,"C":1.38,
             "Q":3.9,"E":6.32,"G":7.03,"H":2.26,"I":5.49,
             "L":9.68,"K":5.19,"M":2.32,"F":3.87,"P":5.02,
             "S":7.14,"T":5.53,"W":1.25,"Y":2.91,"V":6.73} 

charge = {"A":0,"D":-1,"E":-1,"H":+0.1,"C":0,
          "Y":0,"K":+1,"R":+1,"N":0,"Q":0,
          "G":0,"I":0,"L":0,"M":0,"F":0,
          "P":0, "S":0,"T":0,"W":0,"V":0}

vloumn = {"A":88.6,"R":173.4,"N":114.1,"D":111.1,"C":108.5,
          "Q":143.8,"E":138.4,"G":60.1,"H":153.2,"I":166.7,
          "L":166.7,"K":168.6,"M":162.9,"F":189.9,"P":112.7,
          "S":89.0,"T":116.1,"W":227.8,"Y":193.6,"V":140.0}


AA_list = ["Q","W","E","R","T","Y","I","P","A","S","D","F","G","H","K","L","C","V","N","M"]


def context_independent(mutation):
    #mutation in wildnummut,
    wildtype = mutation[0]
    muttype = mutation[-1]
    mutationlist = []
    for aa in AA_list:
        if aa == wildtype:
            mutationlist.append(-1)
        elif aa == muttype:
            mutationlist.append(1)
        else:
            mutationlist.append(0)
    number = mutation[1:-1]
    V_wild = vloumn.get(wildtype)
    V_mut = vloumn.get(muttype)
    Hi_wild = hydropathy_index.get(wildtype)
    Hi_mut = hydropathy_index.get(muttype)
    C_wild = charge.get(wildtype)
    C_mut = charge.get(muttype)
    O_wild = occurance.get(wildtype)
    O_mut = occurance.get(muttype)
    context_independent_list = mutationlist+[V_wild,V_mut,Hi_wild,Hi_mut,C_wild,C_mut,O_wild,O_mut]
    return context_independent_list

In [40]:
import pandas as pd
import numpy as np
import string

rm_lc = str.maketrans(dict.fromkeys(string.ascii_lowercase))

ali_dict = {}


def read_a3m(a3mfilename):
    a3mfile = open(a3mfilename)
    for line in a3mfile:
        if line[0] == ">":
            head = line.strip()
            ali_dict[head] = ""
            #print(line.strip())
        else:
            line = line.translate(rm_lc).strip()
            lst = []
            for x in line:
                lst.append(x)
            ali_dict[head] = lst
            #print(lst)

In [41]:
read_a3m("g.a3m")

In [42]:
df = pd.DataFrame(ali_dict)
seq_length = df.shape[0]
seq_num = df.shape[1]
df_msa = df.T
df_msa.head()



,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
>G,M,T,Y,K,L,I,L,N,G,K,...,D,A,T,K,T,F,T,V,T,E
>UniRef100_Q53337 Protein G IgG Fc binding domain n=1 Tax=Streptococcus sp. CMCC 32138 TaxID=184480 RepID=Q53337_9STRE,T,T,Y,K,L,V,I,N,G,K,...,D,A,T,K,T,F,T,V,T,E
>UniRef100_UPI0000111373 PROTEIN G n=1 Tax=Streptococcus sp. TaxID=1306 RepID=UPI0000111373,T,T,Y,K,L,I,L,N,G,K,...,D,A,T,K,T,F,T,V,T,E
>UniRef100_UPI00001115E7 IMMUNOGLOBULIN G BINDING PROTEIN G n=1 Tax=Streptococcus sp. TaxID=1306 RepID=UPI00001115E7,T,T,F,K,L,I,I,N,G,K,...,D,A,T,K,T,W,T,V,T,E
>UniRef100_UPI0000111875 GB1-C3B4 n=1 Tax=Streptococcus sp. G148 TaxID=1324 RepID=UPI0000111875,T,T,F,K,L,I,I,N,G,K,...,D,A,T,K,T,F,T,V,T,E


In [43]:
alphabet = "ARNDCQEGHILKMFPSTWYV-"
p_mas = np.zeros([20,56])
for p in range(seq_length):
    for num,AA in enumerate(alphabet):
        if AA != "-":
            count = df_msa[p].value_counts().get(AA)
            if count == None:
                p_mas[num][p] = 0
            else:
                p_mas[num][p] = count/seq_num
                #print(num,p,count/seq_num)
    p = p + 1
        #print(num,AA)
    

In [44]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import string
import gremlin

# ==========================================================================
# PREP MSA
# ==========================================================================
# parse fasta
headers, seqs = gremlin.parse_fasta("g.a3m",a3m=True)

train_seqs, test_seqs = gremlin.split_train_test(seqs, frac_test=0.1)

# process input training sequences
msa = gremlin.mk_msa(train_seqs, gap_cutoff=0.5, eff_cutoff=0.8)
# gap_cutoff=0.5 (positions with ≥ 50% gaps are removed)
# eff_cutoff=0.8 (sequences that share ≥ 80% sequence identity 
# are considered "effectively" a single sequence)

#%%time
# ==========================================================================
# RUN GREMLIN
# ==========================================================================
mrf = gremlin.GREMLIN(msa,lam_w=0.1)

# NOTE: lam_v (for one-body term) lam_w (for two-body term) can be used to regularize the model
#
# for contact prediction we find lam_w = 0.01 to be most optimial
# (even though it's technically overfitting on the data!)
# the overfitting is partly corrected by APC
#
# for design/scoring you may want to bump the lam_w to a higher value!



starting 1543.91
iter 10 1739.74
iter 20 1498.47
iter 30 1429.38
iter 40 1416.23
iter 50 1407.1
iter 60 1405.56
iter 70 1404.72
iter 80 1404.35
iter 90 1404.27
iter 100 1404.24


In [45]:
# Y33L
alphabet = "ARNDCQEGHILKMFPSTWYV-"
A2N = {}
for num,AA in enumerate(alphabet):
    A2N[AA]=num

In [46]:
mutation = "Y33L"
def extract_feature(mutation):
    w_num = A2N[mutation[0]]
    m_num = A2N[mutation[-1]]
    pos = int(mutation[1:-1])

    w_mrf_v = mrf["v"].T[w_num][pos-1]
    m_mrf_v = mrf["v"].T[m_num][pos-1]
    w_P = p_mas[w_num][pos-1]
    m_P = p_mas[m_num][pos-1]
    flst = context_independent(mutation)+[w_mrf_v,m_mrf_v,w_P,m_P]
    return flst



In [47]:
dataset = open('G_clean.txt')
target = []
data = {}
for line in dataset:
    #print(line.strip().split())
    mutation = line.strip().split()[0]
    flst = extract_feature(mutation)
    ddg = float(line.strip().split()[1])
    #flst.append(ddg)
    data[mutation] = flst + [ddg]

data

{'N37P': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  114.1,
  112.7,
  -0.78,
  0.12,
  0,
  0,
  3.93,
  5.02,
  4.018893,
  -1.4617897,
  0.6206896551724138,
  0.0,
  -3.694],
 'A20P': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  88.6,
  112.7,
  0.62,
  0.12,
  0,
  0,
  8.76,
  5.02,
  5.8652487,
  -0.8398946,
  0.8620689655172413,
  0.0,
  -3.48],
 'Y33G': [0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  193.6,
  60.1,
  0.26,
  0.48,
  0,
  0,
  2.91,
  7.03,
  3.9603307,
  -1.4772683,
  0.5517241379310345,
  0.0,
  -3.459],
 'V39D': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  140.0,
  111.1,
  1.1,
  -0.9,
  0,
  -1,
  6.73,
  5.49,
  4.2788305,
  -1.1875159,
  0.4827586206896552,
  0.0,
  -3.414],
 'T51D': [0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,


In [48]:
df = pd.DataFrame(data).T

In [49]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
N37P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.12,0.0,0.0,3.93,5.02,4.018893,-1.461790,0.620690,0.0,-3.694
A20P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,...,0.12,0.0,0.0,8.76,5.02,5.865249,-0.839895,0.862069,0.0,-3.480
Y33G,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.48,0.0,0.0,2.91,7.03,3.960331,-1.477268,0.551724,0.0,-3.459
V39D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.90,0.0,-1.0,6.73,5.49,4.278831,-1.187516,0.482759,0.0,-3.414
T51D,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,-0.90,0.0,-1.0,5.53,5.49,4.822475,-1.227427,0.862069,0.0,-3.311


0:31 是data，32是target value

In [50]:
pd.DataFrame.to_csv(df,'Gb1_dataset.csv')

Object `pd.DataFrame.to_csv()` not found.
